In [ ]:
# Install/Upgrade purviewcli package
%pip install --upgrade purviewcli

In [2]:
# Environment Variables
%env PURVIEW_NAME=YOUR_PURVIEW_ACCOUNT
%env AZURE_CLIENT_ID=YOUR_CLIENT_ID
%env AZURE_TENANT_ID=YOUR_TENANT_ID
%env AZURE_CLIENT_SECRET=YOUR_CLIENT_SECRET

env: PURVIEW_NAME=purview-sandbox


In [3]:
# Helper Methods
import json
def getJSON(raw_output):
    output = ''.join(raw_output)
    json_obj = json.loads(output)
    return json_obj

import csv
def export(table):
    with open('output.csv', 'w',newline='', encoding="utf-8") as output:
        csv_writer = csv.writer(output)
        csv_writer.writerows(table)

In [4]:
# 1. Get Sources
print('[INFO] Retrieving Sources...')
data = !pv scan readDatasources
sources = getJSON(data)

filtered_sources = []
counter = 0
for source in sources['value']:
    if source['kind'] != 'Collection':
        counter += 1
        datasource = source['name']
        filtered_sources.append(datasource)
        print('{0}. [{1}] {2}'.format(counter, source['kind'], datasource))
print('[INFO] Complete!')

[INFO] Retrieving Sources...
1. [AdlsGen2] AzureDataLakeStorage-kl2
2. [AdlsGen1] AdlsGen1
3. [AdlsGen2] AdlsGen2
4. [AzureStorage] AzureStorage
5. [AzureSqlDatabase] AzureSqlDatabase
6. [AzureSqlDataWarehouse] AzureSqlDataWarehouse
7. [SqlServerDatabase] SqlServerDatabase
8. [AzureSqlDatabaseManagedInstance] AzureSqlDatabaseManagedInstance
9. [AzureCosmosDb] AzureCosmosDb
10. [AzureDataExplorer] AzureDataExplorer
11. [PowerBI] PowerBI
12. [Teradata] Teradata
13. [SapEcc] SapEcc
14. [SapS4Hana] SapS4Hana
15. [Hive] Hive
16. [AmazonS3] AmazonS3
17. [AzureSqlDatabase] AzureSqlDatabase-JND
18. [Oracle] Oracle-htl
19. [AzureSubscription] AzureResource-S2g
20. [AdlsGen2] AzureDataLakeStorage-omK
21. [AzureStorage] DataSource1
22. [AdlsGen2] DataSource2
[INFO] Complete!


In [5]:
# 2. Get Scans
scans = {}
number_of_sources = len(filtered_sources)
counter = 0

print('[INFO] Retrieving Scans...')
for source in filtered_sources:
    counter += 1
    data2 = !pv scan readScans --dataSourceName {source}
    scan = getJSON(data2)
    print('{0} of {1} (Source: {2}; Scans: {3})'.format(counter, number_of_sources, source, len(scan['value'])))
    scans[source] = scan
print('[INFO] Complete!')

[INFO] Retrieving Scans...
1 of 22 (Source: AzureDataLakeStorage-kl2; Scans: 21)
2 of 22 (Source: AdlsGen1; Scans: 0)
3 of 22 (Source: AdlsGen2; Scans: 0)
4 of 22 (Source: AzureStorage; Scans: 0)
5 of 22 (Source: AzureSqlDatabase; Scans: 0)
6 of 22 (Source: AzureSqlDataWarehouse; Scans: 0)
7 of 22 (Source: SqlServerDatabase; Scans: 0)
8 of 22 (Source: AzureSqlDatabaseManagedInstance; Scans: 0)
9 of 22 (Source: AzureCosmosDb; Scans: 0)
10 of 22 (Source: AzureDataExplorer; Scans: 0)
11 of 22 (Source: PowerBI; Scans: 0)
12 of 22 (Source: Teradata; Scans: 0)
13 of 22 (Source: SapEcc; Scans: 0)
14 of 22 (Source: SapS4Hana; Scans: 0)
15 of 22 (Source: Hive; Scans: 0)
16 of 22 (Source: AmazonS3; Scans: 0)
17 of 22 (Source: AzureSqlDatabase-JND; Scans: 3)
18 of 22 (Source: Oracle-htl; Scans: 0)
19 of 22 (Source: AzureResource-S2g; Scans: 0)
20 of 22 (Source: AzureDataLakeStorage-omK; Scans: 0)
21 of 22 (Source: DataSource1; Scans: 0)
22 of 22 (Source: DataSource2; Scans: 1)
[INFO] Complete!


In [6]:
# 3. Get Scan History
results = []
headers = ["assetsClassified", "assetsDiscovered", "dataSourceType", "endTime", "error", "errorMessage", "id", "parentId", "pipelineStartTime", "queuedTime", "resourceId", "runType", "scanLevelType", "scanRulesetType", "scanRulesetVersion", "startTime", "status", "source", "scanName"]
results.append(headers)
counter = 0

print('[INFO] Retrieving Scan History...')
for datasource in scans:
    counter += 1
    print('[{0} of {1}] Datasource {2} has {3} scans.'.format(
        counter,
        number_of_sources,
        datasource,
        len(scans[datasource]['value'])
        )
    )
    for scan in scans[datasource]['value']:
        print(' - Getting Scan History for Scan: {0}'.format(scan['name']))
        history = !pv scan readScanHistory --dataSourceName {datasource} --scanName {scan['name']}
        history = getJSON(history)
        for record in history['value']:
            record.pop('diagnostics')
            values = list(record.values())
            values.append(datasource)
            values.append(scan['name'])
            results.append(values) 
print('[INFO] Complete!')       

[INFO] Retrieving Scan History...
[1 of 22] Datasource AzureDataLakeStorage-kl2 has 21 scans.
 - Getting Scan History for Scan: Scan-oYy
 - Getting Scan History for Scan: Scan-0Sk
 - Getting Scan History for Scan: Scan-5LW
 - Getting Scan History for Scan: Scan-J2D
 - Getting Scan History for Scan: Scan-2tJ
 - Getting Scan History for Scan: Scan-ier
 - Getting Scan History for Scan: Scan-jOI
 - Getting Scan History for Scan: Scan-blo
 - Getting Scan History for Scan: Scan-Mt7
 - Getting Scan History for Scan: Scan-Joe
 - Getting Scan History for Scan: Scan-myz
 - Getting Scan History for Scan: Scan-RqM
 - Getting Scan History for Scan: Scan-qvF
 - Getting Scan History for Scan: Scan-nEm
 - Getting Scan History for Scan: Scan-Ojs
 - Getting Scan History for Scan: Scan-JCZ
 - Getting Scan History for Scan: Scan-dZY
 - Getting Scan History for Scan: Scan-52q
 - Getting Scan History for Scan: Scan-zqO
 - Getting Scan History for Scan: Scan-HcH
 - Getting Scan History for Scan: Scan-vbs
[2 

In [7]:
# 4. Export to CSV
export(results)
print('[INFO] Scan history has been exported to output.csv')

[INFO] Scan history has been exported to output.csv
